### This notebook train a neural network to predict glucose level.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt

### DateSet

In [ ]:
header = np.array(["Timestamp", "Date", "Lean Meat", "Sauce or Soft Drink with Sugar", "Carbohydrate", "Fried Food",
     "Alcohol Consumption", "Number of Meals", "Minutes of Running", "Minutes of Other Exercise", 
     "Portion Size", "Stress Level", "Sleep Duration", "Sleep Score", "Glucose Level (next morning)"])
data = np.array([
    ["11/25/2024 10:58:34", "11/24/2024", 0, 0, 0, 0, 0, 1, 0, 0, 3, 3, "07:02", 81, 128],
    ["11/26/2024 9:51:53", "11/25/2024", 1, 0, 0, 0, 0, 1, 36, 0, 2, 3, "7:02", 68, 125],
    ["11/27/2024 11:09:58", "11/26/2024", 0, 0, 0, 0, 0, 1, 34, 0, 2, 3, "6:58", 75, 116],
    ["11/28/2024 7:45:19", "11/27/2024", 0, 0, 0, 0, 0, 1, 36, 0, 3, 2, "7:00", 61, 116],
    ["11/29/2024 8:56:11", "11/28/2024", 1, 0, 0, 0, 0, 3, 29, 40, 3, 2, "7:57", 76, 129],
    ["11/30/2024 18:47:43", "11/29/2024", 1, 0, 0, 0, 0, 1, 0, 0, 1, 2, "6:27", 77, 119],
    ["12/1/2024 17:36:09", "11/30/2024", 1, 0, 1, 0, 0, 1, 55, 37, 2, 1, "9:04", 84, 126],
    ["12/2/2024 8:59:54", "12/1/2024", 1, 0, 0, 1, 0, 1, 40, 0, 3, 2, "7:19", 79, 126],
    ["12/3/2024 16:35:05", "12/2/2024", 1, 0, 0, 0, 0, 2, 0, 34, 2, 3, "6:29", 66, 114]
])
print(header.shape)
print(data.shape)

In [ ]:
def time_to_minutes(time_str):
    """Converts a time string in HH:MM format to total minutes.

    Args:
        time_str: The time string in HH:MM format (e.g., "02:30").

    Returns:
        The total number of minutes, or None if the input is invalid.
    """
    try:
        hours, minutes = map(int, time_str.split(':'))
        if 0 <= hours <= 23 and 0 <= minutes <= 59:
            return hours * 60 + minutes
        else:
            return None  # Invalid time format
    except ValueError:
        return None  # Invalid time format

#### organize data

In [ ]:
# convert xx:xx to minutes
print(header[-3])
l = data.shape[0]
for i in range(l):
  time_str = data[i][-3]
  data[i][-3] = time_to_minutes(time_str)
  print(f"{time_str} --> {data[i][-3]}")
# print(data)

In [ ]:
#separate into input and output
#remove time and output from dataset
columns_to_delete = [0, 1, -1]
X = np.delete(data, columns_to_delete, 1)
print(X)
print(f"X.shape {X.shape}")
Y = data[:, -1]
print(Y)
print(f"Y.shape{Y.shape}")

# Convert string array to float NumPy array
X = X.astype(np.float64)
Y = Y.astype(np.float64)

In [ ]:
# separate into training dataset and verify dataset
# 9 --> 8 training and 1 verify
#X training and X verify
Xt = np.delete(X, -1, 0)
Xv = X[-1, :]
print(Xt)
print(Xt.shape)
print(Xv.shape)
Yt = np.delete(Y, -1, 0)
Yv = Y[-1]
print(Yt)
print(Yt.shape)
print(Yv.shape)

In [ ]:
X_train = np.tile(Xt,(1000,1))
# print(X_train)
Y_train= np.tile(Yt,(1,1000)).T
# print(Y_train.T)
print(X_train.shape, Y_train.shape)   

## Tensorflow model

In [ ]:
tf.random.set_seed(1234)
model = Sequential(
  [
    tf.keras.Input(shape=(X_train.shape[1],)),
    Dense(units=3, activation='relu', name='layer1'),
    Dense(units=1, activation='relu', name='layer2')
  ]
)

In [ ]:
model.summary()

In [ ]:
# W count = # of neurons * input_size + # of neurons
L1_num_params = 12 * 3 + 3   # W1 parameters  + b1 parameters
L2_num_params = 3 * 1 + 1   # W2 parameters  + b2 parameters
print("L1 params = ", L1_num_params, ", L2 params = ", L2_num_params  )

In [ ]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1)
print(f"W2{W2.shape}:\n", W2, f"\nb2{b2.shape}:", b2)

In [ ]:
model.compile(
  loss=tf.keras.losses.MeanSquaredError(),
  optimizer=tf.keras.optimizers.Adam(0.01)
)

model.fit(
  X_train, Y_train,
  epochs=20
)

In [ ]:
# Check if the weights are successfully replaced
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

In [ ]:
Xv = np.expand_dims(Xv, axis=0)
print(f"Xv {Xv}\nXv.shape {Xv.shape}")
print(f"Yv {Yv}")
prediction = model.predict(Xv)
print("prediction = \n", prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
prediction = 
 [[114.9897]]


In [ ]:
# 1. Mean Absolute Error (MAE)
mae = np.mean(np.abs(Yv - prediction))
print(f"Mean Absolute Error (MAE): {mae}")

# 2. Mean Squared Error (MSE)
mse = np.mean((Yv - prediction) ** 2)
print(f"Mean Squared Error (MSE): {mse}")

# 3. Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# 4. Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((Yv - prediction) / Yv)) * 100
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")